pyenv local 3.10.11

pyenv exec python -m venv venv

In [43]:
import requests
from bs4 import BeautifulSoup
import re
# import json
# import io
# from collections.abc import Mapping
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from datetime import date

# Web Scrape

## Chase

In [15]:
url = 'https://creditcards.chase.com/all-credit-cards?iCELL=61GN'
headers = {'User-Agent': 'MOZILLA/5.0'}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    print('succeed')
else:
    print(f"Failed to fetch the page. Status code: {response.status_code}")

soup = BeautifulSoup(response.text, 'html.parser')


succeed


In [ ]:
card_containers = soup.find_all('div', class_='cmp-cardsummary__inner-container')

chase_db = [{} for _ in range(len(card_containers))]
required_keys = ["name", "bonus", "reward", "fee", "apr"]
for card in chase_db:
    for key in required_keys:
        card.setdefault(key, "")

for i, card in enumerate(card_containers):
    # Extract card title
    title_tag = card.find('div', class_='cmp-cardsummary__inner-container__title')
    title_text = title_tag.get_text()
    if title_tag:
        if re.search(r"Business", title_text):
            continue #do not get business cards, only personal cards
        match = re.search(r"^\n*\s*(.*?)\s*(?:(?:Credit Card|Card)\s+)?Links to product page", title_text)
        card_name = re.sub(r"[^a-zA-Z0-9\s]", "", match.group(1)).strip() if match else 'N/A'
        chase_db[i]['name'] = card_name

    # Extract details
    glance_div = card.find('div', class_='cmp-cardsummary__inner-container--glance')
    glance_text = glance_div.get_text().strip().replace("AT A GLANCE", "") if glance_div else 'N/A'
    chase_db[i]['reward'] = glance_text

    # Extract offer summary
    offer_div = card.find('div', class_='cmp-cardsummary__inner-container--card-member-offer')
    offer_text = offer_div.get_text(strip=True).replace("NEW CARDMEMBER OFFER", "") if offer_div else 'N/A'
    chase_db[i]['offer'] = offer_text

    # Extract APR
    apr_div = card.find('div', class_='cmp-cardsummary__inner-container--purchase-apr')
    apr_text = apr_div.get_text(strip=True) if apr_div else 'N/A'
    if apr_div:
        apr_values = apr_div.find_all('span', class_='apr-value')
        apr = '-'.join([v.get_text(strip=True) for v in apr_values])
    else:
        apr = '$0'
    chase_db[i]['apr'] = apr

    # Extract Annual Fee
    fee_div = card.find('div', class_='cmp-cardsummary__inner-container--annual-fee')
    fee_text = fee_div.get_text()
    if fee_div:
        match = re.search(r'\$\d+(\.\d{1,2})?', fee_text) #get numbers only
        fee_text = match.group(0) if match else 'N/A'
    chase_db[i]['fee'] = fee_text


## American Express

In [31]:
#Launch website
options = Options()
options.add_argument("--window-size=1920,1080")
# options.add_argument("--headless=new")
options.add_experimental_option("prefs", {"profile.managed_content_settings.images":2}) #not load images

driver = webdriver.Chrome(options=options)
driver.get("https://americanexpress.com/us/credit-cards/?category=all")

#click all the SHOW MORE buttons to get full list of rewards
buttons = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located(
        (By.XPATH, "//button[contains(@aria-label, 'Show More Benefits')]"))
    )
for button in buttons:
    driver.execute_script("arguments[0].click();", button)


In [33]:
cards = driver.find_elements(By.CSS_SELECTOR, "div._cardTileFlexContainer_1esd3_18")

amex_db = [{} for _ in range(len(cards))]
required_keys = ["name", "bonus", "reward", "fee", "apr"]
for card in amex_db:
    for key in required_keys:
        card.setdefault(key, "")

for i, card in enumerate(cards):
    #name
    name = card.find_element(By.CSS_SELECTOR, "span.pad-1-r").text 
    amex_db[i]['name'] = name

    #fee
    try: #if there is fee
        fee = card.find_element(By.CSS_SELECTOR, "p.hidden-sm-down.dls-gray-06.body-3 > span").text[2:]
    except:
        fee = card.find_element(By.CSS_SELECTOR, "p.hidden-sm-down.dls-gray-06.body-3 > strong").text
    amex_db[i]['fee'] = fee 

    #reward
    rewards = card.find_elements(By.CSS_SELECTOR, "b.dls-deep-blue")
    amex_db[i]['reward'] = '; '.join(reward.text.strip() for reward in rewards)
    

    #offer
    ignore_phrases = ['Welcome Offer', 
                      'Limited Time Offer', 
                      'Apply, and if approved:', 
                      "Find out your offer amount",
                      "Accept the Card with your offer"]
    offers = card.find_elements(By.CSS_SELECTOR, "div._vacOfferTitle_1esd3_262 *")
    visible_texts = set()
    for offer in offers:
       if offer.tag_name == "s":
           continue  # Skip strikethrough text
       text = offer.text.strip().replace("†", "")
       if not any(ignore in text for ignore in ignore_phrases):
           visible_texts.add(text)
    amex_db[i]['bonus'] = " ".join(visible_texts)

   #apr
   # Save the handle of the original tab
    original_tab = driver.current_window_handle

    # Click the link that opens a new tab
    link = card.find_element(By.XPATH, ".//a[@aria-label='Rates and Fees']")
    driver.execute_script("arguments[0].scrollIntoView();", link)
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, ".//a[@aria-label='Rates and Fees']")))
    driver.execute_script("arguments[0].click();", link)

    # Wait for the new tab to open and switch to it
    new_tab = [handle for handle in driver.window_handles if handle != original_tab][0]
    driver.switch_to.window(new_tab)

    # Wait for the paragraph element to be present
    para = WebDriverWait(driver, 10).until(
       EC.presence_of_element_located((By.CSS_SELECTOR, "p[aria-describedby='interest-rates-rowheader-0']")))
    apr = para.find_element(By.TAG_NAME, "b")
    amex_db[i]['apr'] = apr.text.strip()

    # Close the tab and switch back to the original
    driver.close()
    driver.switch_to.window(original_tab)

driver.quit()


In [34]:
amex_db

[{'name': 'The Platinum Card®',
  'bonus': ' after you spend $8,000 in purchases on your new Card within the first 6 months of Card Membership. Welcome offers vary and you may not be eligible for an offer. 175,000 Receive the points Spend $8,000 in 6 months ® AS HIGH AS 175,000\nMembership Rewards® points',
  'reward': '5X Membership Rewards® Points with AmexTravel.com; $240 Digital Entertainment Credit; American Express Global Lounge Collection®; $200 Hotel Credit; $200 Uber Cash',
  'fee': '$695†¤',
  'apr': '20.24% to 29.24%,'},
 {'name': 'American Express® Gold Card',
  'bonus': '100,000  after you spend $6,000 in purchases on your new Card within the first 6 months of Card Membership. Welcome offers vary and you may not be eligible for an offer. Receive the points Spend $6,000 in 6 months ® AS HIGH AS 100,000\nMembership Rewards® points',
  'reward': "Earn 4X Points at Restaurants Worldwide; Earn 4X Points at U.S. Supermarkets; Earn 3X Points on Flights; $120 Uber Cash; $84 Dunkin

In [ ]:
#Parse data
cards =  driver.find_elements(By.CSS_SELECTOR, "div._cardTileRightColumn_1esd3_42")

amex_db = [{} for _ in range(len(cards))]
required_keys = ["name", "bonus", "reward", "fee", "apr"]
for card in amex_db:
    for key in required_keys:
        card.setdefault(key, "")

for i, card in enumerate(cards):
    #card name
    amex_db[i]['name'] = card.find("span", class_ = "pad-1-r").text

    #card rewards
    rewards = card.find_all("b", class_ = "dls-deep-blue")
    concatnated_reward = ' '.join(reward.text.strip() for reward in rewards)
    amex_db[i]['reward'] = concatnated_reward

    #card welcome offer
    offer = card.find("div", {"data-qe-id": "OfferTile"})
    #there are two cases of formatting this section
    if offer.find("span", class_="sc_visuallyHidden"):
        time_period = card.select_one("span.sc_color_gray-06.sc_textBody_1").text.strip()
        amex_db[i]['bonus'] = offer.find("span", class_="sc_visuallyHidden").text + time_period
    else:
        amex_db[i]['bonus'] = offer.get_text(separator=' ')

    #card apr
    apr = card.find("p", class_ = 'pad-1-t')
    amex_db[i]['apr'] = apr.text
# driver.quit()


In [50]:
amex_db

[{'name': 'The Platinum Card®',
  'bonus': ' after you spend $8,000 in purchases on your new Card within the first 6 months of Card Membership. Welcome offers vary and you may not be eligible for an offer. 175,000 Receive the points Spend $8,000 in 6 months ® AS HIGH AS 175,000\nMembership Rewards® points',
  'reward': '5X Membership Rewards® Points with AmexTravel.com; $240 Digital Entertainment Credit; American Express Global Lounge Collection®; $200 Hotel Credit; $200 Uber Cash',
  'fee': '$695†¤',
  'apr': '20.24% to 29.24%,'},
 {'name': 'American Express® Gold Card',
  'bonus': '100,000  after you spend $6,000 in purchases on your new Card within the first 6 months of Card Membership. Welcome offers vary and you may not be eligible for an offer. Receive the points Spend $6,000 in 6 months ® AS HIGH AS 100,000\nMembership Rewards® points',
  'reward': "Earn 4X Points at Restaurants Worldwide; Earn 4X Points at U.S. Supermarkets; Earn 3X Points on Flights; $120 Uber Cash; $84 Dunkin

## Citi

In [ ]:
#Launch website
options = Options()
options.add_argument("--window-size=1920,1080")
options.add_argument("--headless=new")
options.add_experimental_option("prefs", {"profile.managed_content_settings.images":2}) #not load images

driver = webdriver.Chrome(options=options)
driver.delete_all_cookies()


We will iteratively select 3 cards to compare => scrape data from this "Compare" table. 
This is because the "Compare" table puts all the cards' details into the same format
| `len(unprocessed)` | Action                                                     | Get Extra Box? | Break? |
| ------------------ | ---------------------------------------------------------- | -------------- | ------ |
| 0                  | Nothing left. No compare.                                  | ✖              | ✅      |
| 1                  | Select this 1 + any previously seen checkbox, compare both | ✅ (re-use one) | ✅      |
| 2                  | Select both and compare                                    | ✖              | ✅      |
| 3                  | Select all 3 and compare                                   | ✖              | ✅      |
| ≥ 4                | Select 3, compare, uncheck, continue to next group         | ✖              | ✖      |


In [46]:
processed_names = set()
citi_db = [{} for _ in range(20)]
required_keys = ["name", "bonus", "reward", "fee", "apr"]
for card in citi_db:
    for key in required_keys:
        card.setdefault(key, "")
round = 0

while True:
    driver.delete_all_cookies()
    # Load the card listing page
    driver.get("https://www.citi.com/credit-cards/compare/view-all-credit-cards?vac=CONTROL")
    time.sleep(3)
    # Wait for checkboxes to appear and filter to visible, enabled ones
    checkboxes = WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'input[type="checkbox"]'))
    )
    checkboxes = [cb for cb in checkboxes if cb.is_displayed() and cb.is_enabled()]
        
    print(len(checkboxes))

    # === Extract unprocessed checkboxes by card name ===
    unprocessed = []
    for cb in checkboxes:
        raw_label = cb.get_attribute("aria-label")
        if not raw_label:
            continue # will reload page again
        name = re.sub(r"^Compare \(\d+\/3\)\s*", "", raw_label).strip()
        if name not in processed_names:
            cb.card_name = name  # Store for later use
            unprocessed.append(cb)

    print(f"Unprocessed remaining: {len(unprocessed)}")
    if len(unprocessed) == 0:
        print("No checkboxes left. Done.")
        break

    # === Select group of checkboxes to compare ===
    group_names = []
    end_after_comparing = False

    if len(unprocessed) == 1:
        # Select 1 unprocessed + 1 previously processed as reference
        cb1 = next((cb for cb in checkboxes if cb.get_attribute("aria-label") and re.sub(r"^Compare \(\d\/3\)\s*", "",
         cb.get_attribute("aria-label")).strip() in processed_names),None)
        cb2 = unprocessed[0]
        for cb in [cb1, cb2]:
            if cb:
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", cb)
                cb.click()
                if cb == cb2:
                    group_names.append(cb.card_name)
        end_after_comparing = True

    elif len(unprocessed) in [2, 3]:
        for cb in unprocessed:
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", cb)
            cb.click()
            group_names.append(cb.card_name)
        end_after_comparing = True

    elif len(unprocessed) >= 4:
        for cb in unprocessed[:3]:
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", cb)
            cb.click()
            group_names.append(cb.card_name)
        end_after_comparing = False

    # === Click "Compare Now" ===
    try:
        compare_btn = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, '//button[normalize-space(text())="Compare now"]'))
        )
    except TimeoutException:
        compare_btn = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, '//a[normalize-space(text())="Compare Now"]'))
        )
    compare_btn.click()

    # Wait for compare results to load
    WebDriverWait(driver, 10).until(
        EC.text_to_be_present_in_element((By.TAG_NAME, "h1"), "Compare Credit Cards")
    )

    time.sleep(2)

     ### ⛏️ SCRAPE DATA 
    
    # name
    card_names = driver.find_elements(By.CSS_SELECTOR, 'h2.card_title.cc-title[id^="selected-credit-card-"]')
    for i, card_name in enumerate(card_names):
        if len(unprocessed) == 1 and i == 0: # then skip the first card
            continue
        raw_html = card_name.get_attribute("innerHTML")
        name = BeautifulSoup(raw_html, "html.parser").text.strip()
        if len(unprocessed) == 1:
            citi_db[i - 1 + round]['name'] = name
        else:
            citi_db[i + round]['name'] = name

    card0 = driver.find_element(By.CSS_SELECTOR, "td.featurecard_0_0.item.ng-star-inserted")
    card1 = driver.find_element(By.CSS_SELECTOR, "td.featurecard_0_1.item.ng-star-inserted")
    card2 = driver.find_element(By.CSS_SELECTOR, "td.featurecard_0_2.item.ng-star-inserted")

    if len(unprocessed) == 1:
        cards = [card1]
    elif len(unprocessed) == 2:
        cards = [card0, card1]
    else:
        cards = [card0, card1, card2]

    
    for i, card in enumerate(cards):
        # offer
        offers = card.find_elements(By.CSS_SELECTOR, 'p.about_content')
        for offer in offers:
            raw_html = offer.get_attribute("innerHTML")
            if 'bonusAmount' in raw_html and 'hurdleAmount' in raw_html:
                bonus = BeautifulSoup(raw_html, "html.parser").text.strip()
                citi_db[i + round]['bonus'] = bonus

    card0 = driver.find_element(By.CSS_SELECTOR, "td.featurecard_1_0.item.ng-star-inserted")
    card1 = driver.find_element(By.CSS_SELECTOR, "td.featurecard_1_1.item.ng-star-inserted")
    card2 = driver.find_element(By.CSS_SELECTOR, "td.featurecard_1_2.item.ng-star-inserted")

    if len(unprocessed) == 1:
        cards = [card1]
    elif len(unprocessed) == 2:
        cards = [card0, card1]
    else:
        cards = [card0, card1, card2]
    
    for i, card in enumerate(cards):
        # Get all rewards containers (one per card)
        rewards = card.find_elements(By.CLASS_NAME, "cardBenefits")
        for reward in rewards:
            html = reward.get_attribute("innerHTML")
            soup = BeautifulSoup(html, "html.parser")
            reward_items = soup.select("li")
            concatnated_reward = ' '.join(item.get_text(separator=" ", strip=True) for item in reward_items)
            citi_db[i + round]['reward'] = concatnated_reward
    
    card0 = driver.find_element(By.CSS_SELECTOR, "td.featurecard_2_0.item.ng-star-inserted")
    card1 = driver.find_element(By.CSS_SELECTOR, "td.featurecard_2_1.item.ng-star-inserted")
    card2 = driver.find_element(By.CSS_SELECTOR, "td.featurecard_2_2.item.ng-star-inserted")

    if len(unprocessed) == 1:
        cards = [card1]
    elif len(unprocessed) == 2:
        cards = [card0, card1]
    else:
        cards = [card0, card1, card2]
    
    for i, card in enumerate(cards):
        # Fees:
        fees = card.find_elements(By.CSS_SELECTOR, "span[data-id^='annualFeeAmount']")
        for fee in fees:   
            html = fee.get_attribute("innerHTML")
            soup = BeautifulSoup(html, "html.parser")
            text = soup.get_text(strip=True)
            citi_db[i + round]['fee'] = text

    card0 = driver.find_element(By.CSS_SELECTOR, "td.featurecard_3_0.item.ng-star-inserted")
    card1 = driver.find_element(By.CSS_SELECTOR, "td.featurecard_3_1.item.ng-star-inserted")
    card2 = driver.find_element(By.CSS_SELECTOR, "td.featurecard_3_2.item.ng-star-inserted")

    if len(unprocessed) == 1:
        cards = [card1]
    elif len(unprocessed) == 2:
        cards = [card0, card1]
    else:
        cards = [card0, card1, card2]
    
    for i, card in enumerate(cards):
        #APR
        aprs = card.find_elements(By.XPATH, "//li[h3[normalize-space()='Purchase Rate']]")
        for apr in aprs:
            try:
                min_apr = apr.find_element(By.CSS_SELECTOR, "span[data-id^='minimumAnnualPercentageRate']").text
                max_apr = apr.find_element(By.CSS_SELECTOR, "span[data-id^='maximumAnnualPercentageRate']").text
                citi_db[i + round]['apr'] = f"{min_apr}-{max_apr}"
            except:
                try:
                    apr_flat = apr.find_element(By.CSS_SELECTOR, "span[data-id^='annualPercentageRateSame']").text
                    citi_db[i + round]['apr'] = f"{apr_flat}"
                except:
                    citi_db[i + round]['apr'] = "$0"

    if end_after_comparing:
        print("Final compare complete. Done.")
        break

    # === Uncheck selected checkboxes by name ===
    driver.get("https://www.citi.com/credit-cards/compare/view-all-credit-cards?vac=CONTROL")
    checkboxes = WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'input[type="checkbox"]'))
    )

    for name in group_names:
        cb = next((cb for cb in checkboxes
                   if cb.is_displayed() and
                   cb.get_attribute("aria-label") and
                   re.sub(r"^Compare \(\d+\/3\)\s*", "", cb.get_attribute("aria-label")).strip() == name), None)
        if cb and cb.is_selected():
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", cb)
            cb.click()
            time.sleep(3)

    # === Mark cards as processed by name ===
    processed_names.update(group_names)
    round += len(group_names)
    # print("✅ Processed cards:", processed_names)
    print(group_names)

driver.quit()


13
Unprocessed remaining: 13
['Citi Strata Premier℠ Credit Card', 'American Airlines AAdvantage® Platinum Select® World Elite Mastercard®', 'Citi® Diamond Preferred® Credit Card']
13
Unprocessed remaining: 10
['Citi Double Cash® Card', 'Citi Simplicity® Credit Card', 'Costco Anywhere Visa® Card by Citi']
13
Unprocessed remaining: 7
['Citi® / AAdvantage® Executive World Elite Mastercard®', 'Citi® / AAdvantage BusinessTM World Elite Mastercard®', 'Citi Custom Cash® Card']
13
Unprocessed remaining: 4
['American Airlines AAdvantage® MileUp® Card', 'AT&T Points Plus® Card from Citi', 'Costco Anywhere Visa® Business Card by Citi']
13
Unprocessed remaining: 1
Final compare complete. Done.


# Insert into SQLite

In [36]:
import sqlite3
import pandas as pd

In [ ]:
conn = sqlite3.connect("cards.db") 
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS cards (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    etl_date TEXT,
    company TEXT,
    name TEXT,
    bonus TEXT,
    rewards TEXT,
    annual_fee TEXT,
    apr TEXT
)
""")

today = date.today().isoformat()

cursor.executemany("""
    INSERT INTO cards (etl_date, company, name, bonus, rewards, annual_fee, apr)
    VALUES (DATE('now'),'CITI', :name, :bonus, :reward, :fee, :apr)
""", citi_db)

cursor.executemany("""
    INSERT INTO cards (etl_date, company, name, bonus, rewards, annual_fee, apr)
    VALUES (DATE('now'),'CHASE', :name, :bonus, :reward, :fee, :apr)
""", chase_db)

conn = sqlite3.connect("cards.db") 
cursor = conn.cursor()
cursor.executemany("""
    INSERT INTO cards (etl_date, company, name, bonus, rewards, annual_fee, apr)
    VALUES (DATE('now'),'AMEX', :name, :bonus, :reward, :fee, :apr)
""", amex_db)

conn.commit()


df = pd.read_sql_query("SELECT * FROM cards", conn)
conn.close()